<a href="https://colab.research.google.com/github/SOUDIPKARMAKAR/Particle-Swarm-Optimisation-in-supply-chain-network/blob/main/PSO_Supply_chain_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import all necessary libraries

In [ ]:
!pip install psopy
import numpy as np
from psopy import minimize
from scipy.optimize import rosen

# Illustrative Example
* Two Plants
* Downstream~Two Demand points
* Two Components~One critical and One Normal
* Upstream~Two Suppliers
###a[0:5] represents making and transportation cost from suppliers to plants;
###a[6:9] represents making and transportation cost from plants to distribution zones, 
###a[10,11] represents making cost of one unit, Binary Variable represents fixed cost
### 12 Decision Variables
### For pandemic scenario extra 12 varaibles
### Minimise Cost Objective Function

#Pre Pandemic Time

In [ ]:
#Cost Vectoor
a=np.array((5,4,4.7,4.2,6,5.8,10,12,10.2,11.6,3.0,3.2,20000,21000))
a.reshape(1,14)

#Objective Function
fun = lambda x,a: np.dot(a[0:12],x)+np.dot(a[12:14],x[10:12]>0)

#Constraint
cons=  ({'type': 'ineq', 'fun': lambda x:  1000-x[0]-x[1]}, #Capacity Constraint for C1 from S1
        {'type': 'ineq', 'fun': lambda x:  1200-x[2]-x[3]},  #Capacity Constraint for C1 from S2
        {'type': 'ineq', 'fun': lambda x:  1500-x[4]-x[5]},  #Capacity Constraint for C3 from S3
        {'type': 'ineq', 'fun': lambda x:  -100+x[6]+x[7]},  #Demand Constraint of D1 from P1 and P2
        {'type': 'ineq', 'fun': lambda x:  -100+x[8]+x[9]},  #Demand Constraint of D2 from P1 and P2
        #{'type': 'ineq', 'fun': lambda x: 2905/np.sum(x[10:12],axis=0)-0.8}, #Lower Limit of Fill rate
        #{'type': 'ineq', 'fun': lambda x: x[10]+x[11]-400}, #Higher Limit of Fill rate
        {'type': 'ineq', 'fun': lambda x: -x[6]-x[8]+x[10]}, #Production of P1 should exceed distribution from P1
        {'type': 'ineq', 'fun': lambda x: -x[7]-x[9]+x[11]}, #Production of P2 should exceed distribution from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[0]+x[2]}, #Supply from S1 and S2 to P1 for C1 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[1]+x[3]}, #Supply from S1 and S2 to P2 for C1 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[4]}, #Supply from from S1 to P1 for C2 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[5]}, #Supply from from S1 to P2 for C2 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: x[0]}, #Decision Variable from S11 to P1 for C7
        {'type': 'ineq', 'fun': lambda x: x[1]}, #Decision Variable from S11 to P2 for C7
        {'type': 'ineq', 'fun': lambda x: x[2]}, #Decision Variable from S1 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[3]}, #Decision Variable from S1 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[4]}, #Decision Variable from S6 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[5]}, #Decision Variable from S6 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[6]}, #Decision Variable from S4 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[7]}, #Decision Variable from S4 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[8]}, #Decision Variable from S7 to P1 for C4
        {'type': 'ineq', 'fun': lambda x: x[9]}, #Decision Variable from S7 to P2 for C4
        {'type': 'ineq', 'fun': lambda x: x[10]}, #Decision Variable from S9 to P1 for C5
        {'type': 'ineq', 'fun': lambda x: x[11]}, #Decision Variable from S9 to P2 for C5
        #{'type': 'ineq', 'fun': lambda x: x[12]} #Covid Variable from S1 to P1 for C1
        #{'type': 'ineq', 'fun': lambda x: x[13]} #Covid Variable from S1 to P2 for C1
        #{'type': 'ineq', 'fun': lambda x: x[14]}, #Covid Variable from S2 to P1 for C1
        #{'type': 'ineq', 'fun': lambda x: x[15]}, #Covid Variable from S2 to P2 for C1
        #{'type': 'ineq', 'fun': lambda x: x[16]}, #Covid Variable from S3 to P1 for C2
        #{'type': 'ineq', 'fun': lambda x: x[17]}, #Covid Variable from S3 to P2 for C2
        #{'type': 'ineq', 'fun': lambda x: x[18]}, #Covid Variable from P1 to D1 
        #{'type': 'ineq', 'fun': lambda x: x[19]}, #Covid Variable from P2 to D1 
        #{'type': 'ineq', 'fun': lambda x: x[20]}, #Covid Variable from P1 to D2
        #{'type': 'ineq', 'fun': lambda x: x[21]}, #Covid Variable from P2 to D2
        #{'type': 'ineq', 'fun': lambda x: x[22]}, #Covid Variable for P1
        #{'type': 'ineq', 'fun': lambda x: x[23]}, #Covid Variable from P2
        )
value=np.array([[0,1000],[0,1000],[0,1000],[0,1000],[0,1500],[0,1500], # Range of each component from Supplier to each plant sequentially
                [0,100],[0,100],[0,100],[0,100],[0,300],[0,300]]) # Range of Distribution Centre from each plant and production capacity of each plant

from psopy import init_feasible
# print(capacity.shape)

# vectorized implimentation---------------
low=value[:,0]
high=value[:,1]

x0=init_feasible(constraints=cons,low=low,high=high, shape=(100,12),max_retries=2000)
print(x0)
res = minimize(fun, x0, args=a, constraints=cons,options={'g_rate': 1., 'l_rate': 1., 'max_velocity': 4.,'stable_iter': 10000,'savefile':'Result.csv'})
print(res)
init_feasible
print('quants:\n')
print(res.x.reshape((2,6)))
q=res.x.reshape(2,6)
print('qunatity from plant 1 & 2:',q[0,],'\n Covid Scenario:',q[1,])


[[421.71005166 496.99493547  24.74026182 ...  31.98910006 239.88908394
  256.77425892]
 [446.17077336 307.28220148 122.84200687 ...  61.13827011 256.89043378
  153.20953879]
 [195.1927154  512.22881411 908.2227771  ...  40.18741829 297.66659802
  289.86441824]
 ...
 [274.82457815  63.53067491 555.60801801 ...  74.58556246 131.11109623
  293.08440991]
 [314.1067108   19.54804277 859.70322027 ...  97.94278369 108.74829202
  149.65554479]
 [481.01789444 500.27296122 258.17722266 ...  53.56170387 190.49593095
  296.15608855]]
    cvec: array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 9.97844325e-07, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])
     fun: 48349.90942301436
   maxcv: 9.9784

#Lockdown 1.0 Scenario

In [ ]:
#Cost Vector
a=np.array((5,4,4.7,4.2,6,5.8,10,12,10.2,11.6,3.0,3.2,20000,21000))
a.reshape(1,14)

#Objective Function
fun = lambda x,a: np.dot(a[0:12],np.multiply(x[0:12],x[12:24]))+np.dot(a[12:14],x[10:12]>0)
#Constraintd
#Constraint
cons=  ({'type': 'ineq', 'fun': lambda x:  1000-x[0]-x[1]}, #Capacity Constraint for C1 from S1
        {'type': 'ineq', 'fun': lambda x:  1200-x[2]-x[3]},  #Capacity Constraint for C1 from S2
        {'type': 'ineq', 'fun': lambda x:  1500-x[4]-x[5]},  #Capacity Constraint for C3 from S3
        {'type': 'ineq', 'fun': lambda x:  -100+x[6]+x[7]},  #Demand Constraint of D1 from P1 and P2
        {'type': 'ineq', 'fun': lambda x:  -100+x[8]+x[9]},  #Demand Constraint of D2 from P1 and P2
        #{'type': 'ineq', 'fun': lambda x: 2905/np.sum(x[10:12],axis=0)-0.8}, #Lower Limit of Fill rate
        #{'type': 'ineq', 'fun': lambda x: x[10]+x[11]-400}, #Higher Limit of Fill rate
        {'type': 'ineq', 'fun': lambda x: -x[6]-x[8]+x[10]}, #Production of P1 should exceed distribution from P1
        {'type': 'ineq', 'fun': lambda x: -x[7]-x[9]+x[11]}, #Production of P2 should exceed distribution from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[0]+x[2]}, #Supply from S1 and S2 to P1 for C1 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[1]+x[3]}, #Supply from S1 and S2 to P2 for C1 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[4]}, #Supply from from S1 to P1 for C2 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[5]}, #Supply from from S1 to P2 for C2 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: x[0]}, #Decision Variable from S11 to P1 for C7
        {'type': 'ineq', 'fun': lambda x: x[1]}, #Decision Variable from S11 to P2 for C7
        {'type': 'ineq', 'fun': lambda x: x[2]}, #Decision Variable from S1 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[3]}, #Decision Variable from S1 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[4]}, #Decision Variable from S6 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[5]}, #Decision Variable from S6 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[6]}, #Decision Variable from S4 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[7]}, #Decision Variable from S4 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[8]}, #Decision Variable from S7 to P1 for C4
        {'type': 'ineq', 'fun': lambda x: x[9]}, #Decision Variable from S7 to P2 for C4
        {'type': 'ineq', 'fun': lambda x: x[10]}, #Decision Variable from S9 to P1 for C5
        {'type': 'ineq', 'fun': lambda x: x[11]}, #Decision Variable from S9 to P2 for C5
        {'type': 'ineq', 'fun': lambda x: x[12]}, #Covid Variable from S1 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[13]},#Covid Variable from S1 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[14]}, #Covid Variable from S2 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[15]}, #Covid Variable from S2 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[16]}, #Covid Variable from S3 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[17]}, #Covid Variable from S3 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[18]}, #Covid Variable from P1 to D1 
        {'type': 'ineq', 'fun': lambda x: x[19]}, #Covid Variable from P2 to D1 
        {'type': 'ineq', 'fun': lambda x: x[20]}, #Covid Variable from P1 to D2
        {'type': 'ineq', 'fun': lambda x: x[21]}, #Covid Variable from P2 to D2
        {'type': 'ineq', 'fun': lambda x: x[22]}, #Covid Variable for P1
        {'type': 'ineq', 'fun': lambda x: x[23]}, #Covid Variable from P2
        )
value=np.array([[0,1000],[0,1000],[0,1000],[0,1000],[0,1500],[0,1500], # Range of each component from Supplier to each plant sequentially
                [0,100],[0,100],[0,100],[0,100],[0,300],[0,300], # Range of Distribution Centre from each plant and production capacity of each plant
                [0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]]) #Covid Constraint values

from psopy import init_feasible
# print(capacity.shape)

# vectorized implimentation---------------
low=value[:,0]
high=value[:,1]

x0=init_feasible(constraints=cons,low=low,high=high, shape=(100,24),max_retries=2000)
print(x0)
res = minimize(fun, x0, args=a, constraints=cons,options={'g_rate': 1., 'l_rate': 1., 'max_velocity': 4.,'stable_iter': 10000,'savefile':'Result0.0.csv'})
print(res)
init_feasible
print('quants:\n')
print(res.x.reshape((2,12)))
q=res.x.reshape(2,12)
print('qunatity from plant 1 & 2:',q[0,],'\n Covid Scenario:',q[1,])


[[2.87634319e+02 4.01392869e+02 2.93846442e+02 ... 8.72187165e-02
  8.43982193e-01 3.16459748e-01]
 [1.08253200e+02 7.61530406e+02 1.99725547e+02 ... 1.31966466e-01
  4.42792178e-01 3.15802289e-01]
 [4.58603059e+02 1.04919995e+02 5.46398925e+02 ... 9.76299701e-02
  2.76152767e-01 6.31634767e-01]
 ...
 [5.77415212e+02 8.22693885e+01 1.08529315e+02 ... 6.05102072e-01
  9.34363036e-01 9.73224362e-01]
 [6.64287003e+01 6.01964207e+02 4.40503929e+02 ... 4.31589752e-01
  2.05300158e-01 1.76063464e-01]
 [6.85822416e+02 3.90499699e+01 4.95534893e+02 ... 7.67733349e-01
  9.99551201e-01 9.34746209e-01]]
    cvec: array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.0000

#Lockdown 2.0 Scenario

In [ ]:
#Cost Vector
a=np.array((5,4,4.7,4.2,6,5.8,10,12,10.2,11.6,3.0,3.2,20000,21000))
a.reshape(1,14)
#Objective
fun = lambda x,a: np.dot(a[0:12],np.multiply(x[0:12],x[12:24]))+np.dot(a[12:14],x[10:12]>0)
#Constraint
cons=  ({'type': 'ineq', 'fun': lambda x:  1000-x[0]-x[1]}, #Capacity Constraint for C1 from S1
        {'type': 'ineq', 'fun': lambda x:  1200-x[2]-x[3]},  #Capacity Constraint for C1 from S2
        {'type': 'ineq', 'fun': lambda x:  1500-x[4]-x[5]},  #Capacity Constraint for C3 from S3
        {'type': 'ineq', 'fun': lambda x:  -100+x[6]+x[7]},  #Demand Constraint of D1 from P1 and P2
        {'type': 'ineq', 'fun': lambda x:  -100+x[8]+x[9]},  #Demand Constraint of D2 from P1 and P2
        #{'type': 'ineq', 'fun': lambda x: 2905/np.sum(x[10:12],axis=0)-0.8}, #Lower Limit of Fill rate
        #{'type': 'ineq', 'fun': lambda x: x[10]+x[11]-400}, #Higher Limit of Fill rate
        {'type': 'ineq', 'fun': lambda x: -x[6]-x[8]+x[10]}, #Production of P1 should exceed distribution from P1
        {'type': 'ineq', 'fun': lambda x: -x[7]-x[9]+x[11]}, #Production of P2 should exceed distribution from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[0]+x[2]}, #Supply from S1 and S2 to P1 for C1 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[1]+x[3]}, #Supply from S1 and S2 to P2 for C1 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[4]}, #Supply from from S1 to P1 for C2 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[5]}, #Supply from from S1 to P2 for C2 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: x[0]}, #Decision Variable from S11 to P1 for C7
        {'type': 'ineq', 'fun': lambda x: x[1]}, #Decision Variable from S11 to P2 for C7
        {'type': 'ineq', 'fun': lambda x: x[2]}, #Decision Variable from S1 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[3]}, #Decision Variable from S1 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[4]}, #Decision Variable from S6 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[5]}, #Decision Variable from S6 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[6]}, #Decision Variable from S4 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[7]}, #Decision Variable from S4 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[8]}, #Decision Variable from S7 to P1 for C4
        {'type': 'ineq', 'fun': lambda x: x[9]}, #Decision Variable from S7 to P2 for C4
        {'type': 'ineq', 'fun': lambda x: x[10]}, #Decision Variable from S9 to P1 for C5
        {'type': 'ineq', 'fun': lambda x: x[11]}, #Decision Variable from S9 to P2 for C5
        {'type': 'ineq', 'fun': lambda x: x[12]-0.2}, #Covid Variable from S1 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[13]-0.2}, #Covid Variable from S1 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[14]-0.2}, #Covid Variable from S2 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[15]-0.2}, #Covid Variable from S2 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[16]-0.2}, #Covid Variable from S3 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[17]-0.2}, #Covid Variable from S3 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[18]-0.2}, #Covid Variable from P1 to D1 
        {'type': 'ineq', 'fun': lambda x: x[19]-0.2}, #Covid Variable from P2 to D1 
        {'type': 'ineq', 'fun': lambda x: x[20]-0.2}, #Covid Variable from P1 to D2
        {'type': 'ineq', 'fun': lambda x: x[21]-0.2}, #Covid Variable from P2 to D2
        {'type': 'ineq', 'fun': lambda x: x[22]-0.2}, #Covid Variable for P1
        {'type': 'ineq', 'fun': lambda x: x[23]-0.2}, #Covid Variable from P2
        )
value=np.array([[0,1000],[0,1000],[0,1000],[0,1000],[0,1500],[0,1500], # Range of each component from Supplier to each plant sequentially
                [0,100],[0,100],[0,100],[0,100],[0,300],[0,300], # Range of Distribution Centre from each plant and production capacity of each plant
                [0.2,1],[0.2,1],[0.2,1],[0.2,1],[0.2,1],[0.2,1],[0.2,1],[0.2,1],[0.2,1],[0.2,1],[0.2,1],[0.2,1]]) #Covid Constraint values

from psopy import init_feasible
# print(capacity.shape)

# vectorized implimentation---------------
low=value[:,0]
high=value[:,1]

x0=init_feasible(constraints=cons,low=low,high=high, shape=(100,24),max_retries=2000)
print(x0)
res = minimize(fun, x0, args=a, constraints=cons,options={'g_rate': 1., 'l_rate': 1., 'max_velocity': 4.,'stable_iter': 10000,'savefile':'Result_0.2.csv'})
print(res)
init_feasible
print('quants:\n')
print(res.x.reshape((2,12)))
q=res.x.reshape(2,12)
print('qunatity from plant 1 & 2:',q[0,],'\n Covid Scenario:',q[1,])


[[9.10993202e+02 1.18426210e+00 2.99003026e+02 ... 6.83045350e-01
  7.31202810e-01 2.16400800e-01]
 [1.93590064e+02 7.71963409e+02 1.52510609e+02 ... 6.64544885e-01
  6.67665269e-01 8.16181486e-01]
 [6.27207815e+02 2.20571114e+02 1.16592674e+02 ... 6.93155283e-01
  6.78921868e-01 9.39222863e-01]
 ...
 [3.98435549e+02 1.40297553e+02 3.96432877e+02 ... 6.92632124e-01
  3.52738430e-01 3.65657411e-01]
 [3.04796605e+02 1.69663690e+02 8.37794616e+02 ... 7.18882712e-01
  8.33283928e-01 4.83444993e-01]
 [2.44379231e+02 2.70632010e+02 1.21222697e+02 ... 5.41843333e-01
  2.77431601e-01 6.83672264e-01]]
    cvec: array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.0000

#Lockdown 3.0 Scenario

In [ ]:
#Cost Vector
a=np.array((5,4,4.7,4.2,6,5.8,10,12,10.2,11.6,3.0,3.2,20000,21000))
a.reshape(1,14)
#Objective
fun = lambda x,a: np.dot(a[0:12],np.multiply(x[0:12],x[12:24]))+np.dot(a[12:14],x[10:12]>0)
#Constraint
cons=  ({'type': 'ineq', 'fun': lambda x:  1000-x[0]-x[1]}, #Capacity Constraint for C1 from S1
        {'type': 'ineq', 'fun': lambda x:  1200-x[2]-x[3]},  #Capacity Constraint for C1 from S2
        {'type': 'ineq', 'fun': lambda x:  1500-x[4]-x[5]},  #Capacity Constraint for C3 from S3
        {'type': 'ineq', 'fun': lambda x:  -100+x[6]+x[7]},  #Demand Constraint of D1 from P1 and P2
        {'type': 'ineq', 'fun': lambda x:  -100+x[8]+x[9]},  #Demand Constraint of D2 from P1 and P2
        #{'type': 'ineq', 'fun': lambda x: 2905/np.sum(x[10:12],axis=0)-0.8}, #Lower Limit of Fill rate
        #{'type': 'ineq', 'fun': lambda x: x[10]+x[11]-400}, #Higher Limit of Fill rate
        {'type': 'ineq', 'fun': lambda x: -x[6]-x[8]+x[10]}, #Production of P1 should exceed distribution from P1
        {'type': 'ineq', 'fun': lambda x: -x[7]-x[9]+x[11]}, #Production of P2 should exceed distribution from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[0]+x[2]}, #Supply from S1 and S2 to P1 for C1 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[1]+x[3]}, #Supply from S1 and S2 to P2 for C1 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[4]}, #Supply from from S1 to P1 for C2 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[5]}, #Supply from from S1 to P2 for C2 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: x[0]}, #Decision Variable from S11 to P1 for C7
        {'type': 'ineq', 'fun': lambda x: x[1]}, #Decision Variable from S11 to P2 for C7
        {'type': 'ineq', 'fun': lambda x: x[2]}, #Decision Variable from S1 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[3]}, #Decision Variable from S1 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[4]}, #Decision Variable from S6 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[5]}, #Decision Variable from S6 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[6]}, #Decision Variable from S4 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[7]}, #Decision Variable from S4 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[8]}, #Decision Variable from S7 to P1 for C4
        {'type': 'ineq', 'fun': lambda x: x[9]}, #Decision Variable from S7 to P2 for C4
        {'type': 'ineq', 'fun': lambda x: x[10]}, #Decision Variable from S9 to P1 for C5
        {'type': 'ineq', 'fun': lambda x: x[11]}, #Decision Variable from S9 to P2 for C5
        {'type': 'ineq', 'fun': lambda x: x[12]-0.25}, #Covid Variable from S1 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[13]-0.25}, #Covid Variable from S1 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[14]-0.25}, #Covid Variable from S2 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[15]-0.25}, #Covid Variable from S2 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[16]-0.25}, #Covid Variable from S3 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[17]-0.25}, #Covid Variable from S3 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[18]-0.25}, #Covid Variable from P1 to D1 
        {'type': 'ineq', 'fun': lambda x: x[19]-0.25}, #Covid Variable from P2 to D1 
        {'type': 'ineq', 'fun': lambda x: x[20]-0.25}, #Covid Variable from P1 to D2
        {'type': 'ineq', 'fun': lambda x: x[21]-0.25}, #Covid Variable from P2 to D2
        {'type': 'ineq', 'fun': lambda x: x[22]-0.25}, #Covid Variable for P1
        {'type': 'ineq', 'fun': lambda x: x[23]-0.25}, #Covid Variable from P2
        )
value=np.array([[0,1000],[0,1000],[0,1000],[0,1000],[0,1500],[0,1500], # Range of each component from Supplier to each plant sequentially
                [0,100],[0,100],[0,100],[0,100],[0,300],[0,300], # Range of Distribution Centre from each plant and production capacity of each plant
                [0.25,1],[0.25,1],[0.25,1],[0.25,1],[0.25,1],[0.25,1],[0.25,1],[0.25,1],[0.25,1],[0.25,1],[0.25,1],[0.25,1]]) #Covid Constraint values

from psopy import init_feasible
# print(capacity.shape)

# vectorized implimentation---------------
low=value[:,0]
high=value[:,1]

x0=init_feasible(constraints=cons,low=low,high=high, shape=(100,24),max_retries=2000)
print(x0)
res = minimize(fun, x0, args=a, constraints=cons,options={'g_rate': 1., 'l_rate': 1., 'max_velocity': 4.,'stable_iter': 10000,'savefile':'Result0.25.csv'})
print(res)
init_feasible
print('quants:\n')
print(res.x.reshape((2,12)))
q=res.x.reshape(2,12)
print('qunatity from plant 1 & 2:',q[0,],'\n Covid Scenario:',q[1,])


[[8.31346868e+02 6.93611352e+01 1.17413527e+02 ... 6.51501770e-01
  7.43014014e-01 4.34113953e-01]
 [6.17281361e+02 3.30368335e+02 2.58954505e+02 ... 4.84666320e-01
  8.19405944e-01 4.85785059e-01]
 [2.74393084e+02 6.75661918e+02 5.92450019e+02 ... 9.25220606e-01
  2.53765952e-01 8.17648409e-01]
 ...
 [8.13481332e+02 1.25961090e+02 3.25812735e+02 ... 2.77469832e-01
  3.88345566e-01 6.49832082e-01]
 [5.15775952e+02 2.74468565e+02 4.94436657e+02 ... 4.27099162e-01
  6.96291026e-01 4.05348841e-01]
 [3.07588720e+02 1.25528261e+02 7.02292666e+02 ... 3.51848965e-01
  6.91915629e-01 4.80223901e-01]]
    cvec: array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.0000

#Lockdown 4.0 Scenario


In [ ]:
#Cost Vector
a=np.array((5,4,4.7,4.2,6,5.8,10,12,10.2,11.6,3.0,3.2,20000,21000))
a.reshape(1,14)
#Objective
fun = lambda x,a: np.dot(a[0:12],np.multiply(x[0:12],x[12:24]))+np.dot(a[12:14],x[10:12]>0)
#Constraint
cons=  ({'type': 'ineq', 'fun': lambda x:  1000-x[0]-x[1]}, #Capacity Constraint for C1 from S1
        {'type': 'ineq', 'fun': lambda x:  1200-x[2]-x[3]},  #Capacity Constraint for C1 from S2
        {'type': 'ineq', 'fun': lambda x:  1500-x[4]-x[5]},  #Capacity Constraint for C3 from S3
        {'type': 'ineq', 'fun': lambda x:  -100+x[6]+x[7]},  #Demand Constraint of D1 from P1 and P2
        {'type': 'ineq', 'fun': lambda x:  -100+x[8]+x[9]},  #Demand Constraint of D2 from P1 and P2
        #{'type': 'ineq', 'fun': lambda x: 2905/np.sum(x[10:12],axis=0)-0.8}, #Lower Limit of Fill rate
        #{'type': 'ineq', 'fun': lambda x: x[10]+x[11]-400}, #Higher Limit of Fill rate
        {'type': 'ineq', 'fun': lambda x: -x[6]-x[8]+x[10]}, #Production of P1 should exceed distribution from P1
        {'type': 'ineq', 'fun': lambda x: -x[7]-x[9]+x[11]}, #Production of P2 should exceed distribution from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[0]+x[2]}, #Supply from S1 and S2 to P1 for C1 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[1]+x[3]}, #Supply from S1 and S2 to P2 for C1 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[4]}, #Supply from from S1 to P1 for C2 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[5]}, #Supply from from S1 to P2 for C2 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: x[0]}, #Decision Variable from S11 to P1 for C7
        {'type': 'ineq', 'fun': lambda x: x[1]}, #Decision Variable from S11 to P2 for C7
        {'type': 'ineq', 'fun': lambda x: x[2]}, #Decision Variable from S1 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[3]}, #Decision Variable from S1 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[4]}, #Decision Variable from S6 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[5]}, #Decision Variable from S6 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[6]}, #Decision Variable from S4 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[7]}, #Decision Variable from S4 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[8]}, #Decision Variable from S7 to P1 for C4
        {'type': 'ineq', 'fun': lambda x: x[9]}, #Decision Variable from S7 to P2 for C4
        {'type': 'ineq', 'fun': lambda x: x[10]}, #Decision Variable from S9 to P1 for C5
        {'type': 'ineq', 'fun': lambda x: x[11]}, #Decision Variable from S9 to P2 for C5
        {'type': 'ineq', 'fun': lambda x: x[12]-0.3}, #Covid Variable from S1 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[13]-0.3}, #Covid Variable from S1 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[14]-0.3}, #Covid Variable from S2 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[15]-0.3}, #Covid Variable from S2 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[16]-0.3}, #Covid Variable from S3 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[17]-0.3}, #Covid Variable from S3 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[18]-0.3}, #Covid Variable from P1 to D1 
        {'type': 'ineq', 'fun': lambda x: x[19]-0.3}, #Covid Variable from P2 to D1 
        {'type': 'ineq', 'fun': lambda x: x[20]-0.3}, #Covid Variable from P1 to D2
        {'type': 'ineq', 'fun': lambda x: x[21]-0.3}, #Covid Variable from P2 to D2
        {'type': 'ineq', 'fun': lambda x: x[22]-0.3}, #Covid Variable for P1
        {'type': 'ineq', 'fun': lambda x: x[23]-0.3}, #Covid Variable from P2
        )
value=np.array([[0,1000],[0,1000],[0,1000],[0,1000],[0,1500],[0,1500], # Range of each component from Supplier to each plant sequentially
                [0,100],[0,100],[0,100],[0,100],[0,300],[0,300], # Range of Distribution Centre from each plant and production capacity of each plant
                [0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1],[0.3,1]]) #Covid Constraint values

from psopy import init_feasible
# print(capacity.shape)

# vectorized implimentation---------------
low=value[:,0]
high=value[:,1]

x0=init_feasible(constraints=cons,low=low,high=high, shape=(100,24),max_retries=2000)
print(x0)
res = minimize(fun, x0, args=a, constraints=cons,options={'g_rate': 1., 'l_rate': 1., 'max_velocity': 4.,'stable_iter': 10000,'savefile':'Result0.3.csv'})
print(res)
init_feasible
print('quants:\n')
print(res.x.reshape((2,12)))
q=res.x.reshape(2,12)
print('qunatity from plant 1:',q[0,],'\n Covid Scenario:',q[1,])


[[3.81934077e+02 4.92543292e+02 5.27589427e+01 ... 3.74954749e-01
  3.64093610e-01 7.90168071e-01]
 [7.85241756e+02 1.89450689e+02 5.58657592e+02 ... 4.14266150e-01
  8.82601989e-01 4.24317900e-01]
 [4.23291090e+02 3.59940108e+02 1.39469326e+01 ... 5.72282841e-01
  3.19869331e-01 3.09746205e-01]
 ...
 [1.37835763e+02 3.96115955e+02 5.96589914e+02 ... 5.13400741e-01
  3.51353244e-01 3.52613410e-01]
 [2.05208383e+02 1.69555439e+02 1.40576586e+02 ... 5.34130041e-01
  6.68481524e-01 7.90172314e-01]
 [4.99286745e+02 3.86719024e+02 3.98551549e+02 ... 9.85429660e-01
  7.01924651e-01 9.09543372e-01]]
    cvec: array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.0000

#Unlock 1.0 Scenario

In [ ]:
#Cost Vector
a=np.array((5,4,4.7,4.2,6,5.8,10,12,10.2,11.6,3.0,3.2,20000,21000))
a.reshape(1,14)
#Objective
fun = lambda x,a: np.dot(a[0:12],np.multiply(x[0:12],x[12:24]))+np.dot(a[12:14],x[10:12]>0)
#Constraint
cons=  ({'type': 'ineq', 'fun': lambda x:  1000-x[0]-x[1]}, #Capacity Constraint for C1 from S1
        {'type': 'ineq', 'fun': lambda x:  1200-x[2]-x[3]},  #Capacity Constraint for C1 from S2
        {'type': 'ineq', 'fun': lambda x:  1500-x[4]-x[5]},  #Capacity Constraint for C3 from S3
        {'type': 'ineq', 'fun': lambda x:  -100+x[6]+x[7]},  #Demand Constraint of D1 from P1 and P2
        {'type': 'ineq', 'fun': lambda x:  -100+x[8]+x[9]},  #Demand Constraint of D2 from P1 and P2
        #{'type': 'ineq', 'fun': lambda x: 2905/np.sum(x[10:12],axis=0)-0.8}, #Lower Limit of Fill rate
        #{'type': 'ineq', 'fun': lambda x: x[10]+x[11]-400}, #Higher Limit of Fill rate
        {'type': 'ineq', 'fun': lambda x: -x[6]-x[8]+x[10]}, #Production of P1 should exceed distribution from P1
        {'type': 'ineq', 'fun': lambda x: -x[7]-x[9]+x[11]}, #Production of P2 should exceed distribution from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[0]+x[2]}, #Supply from S1 and S2 to P1 for C1 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[1]+x[3]}, #Supply from S1 and S2 to P2 for C1 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[4]}, #Supply from from S1 to P1 for C2 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[5]}, #Supply from from S1 to P2 for C2 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: x[0]}, #Decision Variable from S11 to P1 for C7
        {'type': 'ineq', 'fun': lambda x: x[1]}, #Decision Variable from S11 to P2 for C7
        {'type': 'ineq', 'fun': lambda x: x[2]}, #Decision Variable from S1 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[3]}, #Decision Variable from S1 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[4]}, #Decision Variable from S6 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[5]}, #Decision Variable from S6 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[6]}, #Decision Variable from S4 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[7]}, #Decision Variable from S4 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[8]}, #Decision Variable from S7 to P1 for C4
        {'type': 'ineq', 'fun': lambda x: x[9]}, #Decision Variable from S7 to P2 for C4
        {'type': 'ineq', 'fun': lambda x: x[10]}, #Decision Variable from S9 to P1 for C5
        {'type': 'ineq', 'fun': lambda x: x[11]}, #Decision Variable from S9 to P2 for C5
        {'type': 'ineq', 'fun': lambda x: x[12]-0.4}, #Covid Variable from S1 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[13]-0.4}, #Covid Variable from S1 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[14]-0.4}, #Covid Variable from S2 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[15]-0.4}, #Covid Variable from S2 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[16]-0.4}, #Covid Variable from S3 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[17]-0.4}, #Covid Variable from S3 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[18]-0.4}, #Covid Variable from P1 to D1 
        {'type': 'ineq', 'fun': lambda x: x[19]-0.4}, #Covid Variable from P2 to D1 
        {'type': 'ineq', 'fun': lambda x: x[20]-0.4}, #Covid Variable from P1 to D2
        {'type': 'ineq', 'fun': lambda x: x[21]-0.4}, #Covid Variable from P2 to D2
        {'type': 'ineq', 'fun': lambda x: x[22]-0.4}, #Covid Variable for P1
        {'type': 'ineq', 'fun': lambda x: x[23]-0.4}, #Covid Variable from P2
        )
value=np.array([[0,1000],[0,1000],[0,1000],[0,1000],[0,1500],[0,1500], # Range of each component from Supplier to each plant sequentially
                [0,100],[0,100],[0,100],[0,100],[0,300],[0,300], # Range of Distribution Centre from each plant and production capacity of each plant
                [0.4,1],[0.4,1],[0.4,1],[0.4,1],[0.4,1],[0.4,1],[0.4,1],[0.4,1],[0.4,1],[0.4,1],[0.4,1],[0.4,1]]) #Covid Constraint values

from psopy import init_feasible
# print(capacity.shape)

# vectorized implimentation---------------
low=value[:,0]
high=value[:,1]

x0=init_feasible(constraints=cons,low=low,high=high, shape=(100,24),max_retries=2000)
print(x0)
res = minimize(fun, x0, args=a, constraints=cons,options={'g_rate': 1., 'l_rate': 1., 'max_velocity': 4.,'stable_iter': 1000,'savefile':'Result0.4.csv'})
print(res)
init_feasible
print('quants:\n')
print(res.x.reshape((2,12)))
q=res.x.reshape(2,12)
print('qunatity from plant 1 & 2:',q[0,],'\n Covid Scenario:',q[1,])


[[6.00067342e+02 2.68846498e+02 4.71533070e+02 ... 7.40169217e-01
  4.61755047e-01 6.57326177e-01]
 [1.12635954e+02 4.63163687e+02 3.76066982e+02 ... 5.43402319e-01
  4.58618897e-01 9.86943489e-01]
 [3.40443604e+02 4.52839510e+02 3.03379612e+02 ... 8.88591265e-01
  4.21423502e-01 9.47243903e-01]
 ...
 [6.10963170e+02 1.12955897e+02 2.43947469e+02 ... 9.87227368e-01
  7.03775517e-01 8.67077167e-01]
 [1.75904904e+01 8.30916765e+02 2.37589866e+02 ... 6.86686707e-01
  7.47957056e-01 9.39154675e-01]
 [4.93561779e+02 2.44300091e+02 1.58632010e+02 ... 6.90107883e-01
  9.87616946e-01 4.85931335e-01]]
    cvec: array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        8.03069078e-07, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.0000

#Unlock 2.0 Scenario

In [ ]:
#Cost Vector
a=np.array((5,4,4.7,4.2,6,5.8,10,12,10.2,11.6,3.0,3.2,20000,21000))
a.reshape(1,14)
#Objective
fun = lambda x,a: np.dot(a[0:12],np.multiply(x[0:12],x[12:24]))+np.dot(a[12:14],x[10:12]>0)
#Constraint
cons=  ({'type': 'ineq', 'fun': lambda x:  1000-x[0]-x[1]}, #Capacity Constraint for C1 from S1
        {'type': 'ineq', 'fun': lambda x:  1200-x[2]-x[3]},  #Capacity Constraint for C1 from S2
        {'type': 'ineq', 'fun': lambda x:  1500-x[4]-x[5]},  #Capacity Constraint for C3 from S3
        {'type': 'ineq', 'fun': lambda x:  -100+x[6]+x[7]},  #Demand Constraint of D1 from P1 and P2
        {'type': 'ineq', 'fun': lambda x:  -100+x[8]+x[9]},  #Demand Constraint of D2 from P1 and P2
        #{'type': 'ineq', 'fun': lambda x: 2905/np.sum(x[10:12],axis=0)-0.8}, #Lower Limit of Fill rate
        #{'type': 'ineq', 'fun': lambda x: x[10]+x[11]-400}, #Higher Limit of Fill rate
        {'type': 'ineq', 'fun': lambda x: -x[6]-x[8]+x[10]}, #Production of P1 should exceed distribution from P1
        {'type': 'ineq', 'fun': lambda x: -x[7]-x[9]+x[11]}, #Production of P2 should exceed distribution from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[0]+x[2]}, #Supply from S1 and S2 to P1 for C1 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[1]+x[3]}, #Supply from S1 and S2 to P2 for C1 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[4]}, #Supply from from S1 to P1 for C2 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[5]}, #Supply from from S1 to P2 for C2 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: x[0]}, #Decision Variable from S11 to P1 for C7
        {'type': 'ineq', 'fun': lambda x: x[1]}, #Decision Variable from S11 to P2 for C7
        {'type': 'ineq', 'fun': lambda x: x[2]}, #Decision Variable from S1 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[3]}, #Decision Variable from S1 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[4]}, #Decision Variable from S6 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[5]}, #Decision Variable from S6 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[6]}, #Decision Variable from S4 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[7]}, #Decision Variable from S4 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[8]}, #Decision Variable from S7 to P1 for C4
        {'type': 'ineq', 'fun': lambda x: x[9]}, #Decision Variable from S7 to P2 for C4
        {'type': 'ineq', 'fun': lambda x: x[10]}, #Decision Variable from S9 to P1 for C5
        {'type': 'ineq', 'fun': lambda x: x[11]}, #Decision Variable from S9 to P2 for C5
        {'type': 'ineq', 'fun': lambda x: x[12]-0.5}, #Covid Variable from S1 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[13]-0.5}, #Covid Variable from S1 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[14]-0.5}, #Covid Variable from S2 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[15]-0.5}, #Covid Variable from S2 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[16]-0.5}, #Covid Variable from S3 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[17]-0.5}, #Covid Variable from S3 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[18]-0.5}, #Covid Variable from P1 to D1 
        {'type': 'ineq', 'fun': lambda x: x[19]-0.5}, #Covid Variable from P2 to D1 
        {'type': 'ineq', 'fun': lambda x: x[20]-0.5}, #Covid Variable from P1 to D2
        {'type': 'ineq', 'fun': lambda x: x[21]-0.5}, #Covid Variable from P2 to D2
        {'type': 'ineq', 'fun': lambda x: x[22]-0.5}, #Covid Variable for P1
        {'type': 'ineq', 'fun': lambda x: x[23]-0.5}, #Covid Variable from P2
        )
value=np.array([[0,1000],[0,1000],[0,1000],[0,1000],[0,1500],[0,1500], # Range of each component from Supplier to each plant sequentially
                [0,100],[0,100],[0,100],[0,100],[0,300],[0,300], # Range of Distribution Centre from each plant and production capacity of each plant
                [0.5,1],[0.5,1],[0.5,1],[0.5,1],[0.5,1],[0.5,1],[0.5,1],[0.5,1],[0.5,1],[0.5,1],[0.5,1],[0.5,1]]) #Covid Constraint values

from psopy import init_feasible
# print(capacity.shape)

# vectorized implimentation---------------
low=value[:,0]
high=value[:,1]

x0=init_feasible(constraints=cons,low=low,high=high, shape=(100,24),max_retries=2000)
print(x0)
res = minimize(fun, x0, args=a, constraints=cons,options={'g_rate': 1., 'l_rate': 1., 'max_velocity': 4.,'stable_iter': 10000,'savefile':'Result0.5.csv'})
print(res)
init_feasible
print('quants:\n')
print(res.x.reshape((2,12)))
q=res.x.reshape(2,12)
print('qunatity from plant 1 & 2:',q[0,],'\n Covid Scenario:',q[1,])


[[6.50220349e+02 1.05377101e+02 3.88632427e+02 ... 5.86450653e-01
  6.64675186e-01 6.24000684e-01]
 [2.34146559e+02 6.19726134e+02 5.80022259e+01 ... 8.00591560e-01
  5.65627212e-01 7.35989119e-01]
 [1.23224800e+02 2.25082793e+02 7.77121294e+02 ... 9.98325509e-01
  6.93305193e-01 6.37892822e-01]
 ...
 [1.57203528e+02 2.71955924e+02 1.27538146e+02 ... 5.03078090e-01
  7.07464572e-01 6.33312629e-01]
 [5.88752862e+02 3.02814132e+02 2.91581354e+02 ... 7.61232974e-01
  6.73381097e-01 8.75778769e-01]
 [3.64500898e+02 2.03371581e+02 8.37674059e+02 ... 5.24481449e-01
  7.90925011e-01 9.19927255e-01]]
    cvec: array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.0000

#Unlock 3.0 Scenario

In [ ]:
#Cost Vector
a=np.array((5,4,4.7,4.2,6,5.8,10,12,10.2,11.6,3.0,3.2,20000,21000))
a.reshape(1,14)
#Objective
fun = lambda x,a: np.dot(a[0:12],np.multiply(x[0:12],x[12:24]))+np.dot(a[12:14],x[10:12]>0)
#Constraint
cons=  ({'type': 'ineq', 'fun': lambda x:  1000-x[0]-x[1]}, #Capacity Constraint for C1 from S1
        {'type': 'ineq', 'fun': lambda x:  1200-x[2]-x[3]},  #Capacity Constraint for C1 from S2
        {'type': 'ineq', 'fun': lambda x:  1500-x[4]-x[5]},  #Capacity Constraint for C3 from S3
        {'type': 'ineq', 'fun': lambda x:  -100+x[6]+x[7]},  #Demand Constraint of D1 from P1 and P2
        {'type': 'ineq', 'fun': lambda x:  -100+x[8]+x[9]},  #Demand Constraint of D2 from P1 and P2
        {'type': 'ineq', 'fun': lambda x: 2905/np.sum(x[10:12],axis=0)-0.8}, #Lower Limit of Fill rate
        #{'type': 'ineq', 'fun': lambda x: x[10]+x[11]-400}, #Higher Limit of Fill rate
        {'type': 'ineq', 'fun': lambda x: -x[6]-x[8]+x[10]}, #Production of P1 should exceed distribution from P1
        {'type': 'ineq', 'fun': lambda x: -x[7]-x[9]+x[11]}, #Production of P2 should exceed distribution from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[0]+x[2]}, #Supply from S1 and S2 to P1 for C1 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[1]+x[3]}, #Supply from S1 and S2 to P2 for C1 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[4]}, #Supply from from S1 to P1 for C2 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[5]}, #Supply from from S1 to P2 for C2 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: x[0]}, #Decision Variable from S11 to P1 for C7
        {'type': 'ineq', 'fun': lambda x: x[1]}, #Decision Variable from S11 to P2 for C7
        {'type': 'ineq', 'fun': lambda x: x[2]}, #Decision Variable from S1 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[3]}, #Decision Variable from S1 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[4]}, #Decision Variable from S6 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[5]}, #Decision Variable from S6 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[6]}, #Decision Variable from S4 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[7]}, #Decision Variable from S4 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[8]}, #Decision Variable from S7 to P1 for C4
        {'type': 'ineq', 'fun': lambda x: x[9]}, #Decision Variable from S7 to P2 for C4
        {'type': 'ineq', 'fun': lambda x: x[10]}, #Decision Variable from S9 to P1 for C5
        {'type': 'ineq', 'fun': lambda x: x[11]}, #Decision Variable from S9 to P2 for C5
        {'type': 'ineq', 'fun': lambda x: x[12]-0.8}, #Covid Variable from S1 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[13]-0.8}, #Covid Variable from S1 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[14]-0.8}, #Covid Variable from S2 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[15]-0.8}, #Covid Variable from S2 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[16]-0.8}, #Covid Variable from S3 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[17]-0.8}, #Covid Variable from S3 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[18]-0.8}, #Covid Variable from P1 to D1 
        {'type': 'ineq', 'fun': lambda x: x[19]-0.8}, #Covid Variable from P2 to D1 
        {'type': 'ineq', 'fun': lambda x: x[20]-0.8}, #Covid Variable from P1 to D2
        {'type': 'ineq', 'fun': lambda x: x[21]-0.8}, #Covid Variable from P2 to D2
        {'type': 'ineq', 'fun': lambda x: x[22]-0.8}, #Covid Variable for P1
        {'type': 'ineq', 'fun': lambda x: x[23]-0.8}, #Covid Variable from P2
        )
value=np.array([[0,1000],[0,1000],[0,1000],[0,1000],[0,1500],[0,1500], # Range of each component from Supplier to each plant sequentially
                [0,100],[0,100],[0,100],[0,100],[0,300],[0,300], # Range of Distribution Centre from each plant and production capacity of each plant
                [0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1]]) #Covid Constraint values

from psopy import init_feasible
# print(capacity.shape)

# vectorized implimentation---------------
low=value[:,0]
high=value[:,1]

x0=init_feasible(constraints=cons,low=low,high=high, shape=(100,24),max_retries=2000)
print(x0)
res = minimize(fun, x0, args=a, constraints=cons,options={'g_rate': 1., 'l_rate': 1., 'max_velocity': 4.,'stable_iter': 10000,'savefile':'Result0.8.csv'})
print(res)
init_feasible
print('quants:\n')
print(res.x.reshape((2,12)))
q=res.x.reshape(2,12)
print('qunatity from plant 1:',q[0,],'\n quantity from plant 2:',q[1,])


[[ 65.79339055  68.99457455 792.5949454  ...   0.9026821    0.97501587
    0.89544094]
 [542.67801934  90.219148   278.90517768 ...   0.8417481    0.81849697
    0.92896903]
 [ 16.33529768 691.03967495 532.95407861 ...   0.82613384   0.86583644
    0.92248226]
 ...
 [ 19.74984638 384.79134542 489.57903439 ...   0.85711388   0.83857654
    0.89684843]
 [ 44.09217514 239.84986816 318.88017644 ...   0.90060921   0.86827496
    0.93249563]
 [179.9101265  550.41116191 214.22055044 ...   0.96295098   0.84138287
    0.80745358]]
    cvec: array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 9.99819719e-07, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.0000

# genralized form

In [ ]:
#Cost Vector
a=np.array((5,4,4.7,4.2,6,5.8,10,12,10.2,11.6,3.0,3.2,20000,21000))
a.reshape(1,14)
variables=25
value=np.array([[0,1000],[0,1000],[0,1000],[0,1000],[0,1500],[0,1500], # Range of each component from Supplier to each plant sequentially
                [0,100],[0,100],[0,100],[0,100],[0,300],[0,300], # Range of Distribution Centre from each plant and production capacity of each plant
                [0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1],[0.8,1]]) #Covid Constraint values
low=value[:,0]
high=value[:,1]
ranges={'quants':range(0,13),}
#Objective
fun = lambda x,a: np.dot(a[0:12],np.multiply(x[0:12],x[12:24]))+np.dot(a[12:14],x[10:12]>0)
#Constraint
cons=[eval("{'type': 'ineq', 'args':[index,lower_limit], 'fun': lambda X,ind,lower_limit: X[ind]-lower_limit[ind] }",{'index':index,'lower_limit':low})  for index in range(0,variables)]
[cons.append(eval("{'type': 'ineq', 'args': [index,higher_limit],'fun': lambda X,ind,higher_limit: higher_limit[ind]- X[ind] }",{'index': index,'higher_limit':high})) for index in range(0,variables)]


cons=  ({'type': 'ineq', 'fun': lambda x:  1000-x[0]-x[1]}, #Capacity Constraint for C1 from S1
        {'type': 'ineq', 'fun': lambda x:  1200-x[2]-x[3]},  #Capacity Constraint for C1 from S2
        {'type': 'ineq', 'fun': lambda x:  1500-x[4]-x[5]},  #Capacity Constraint for C3 from S3
        {'type': 'ineq', 'fun': lambda x:  -100+x[6]+x[7]},  #Demand Constraint of D1 from P1 and P2
        {'type': 'ineq', 'fun': lambda x:  -100+x[8]+x[9]},  #Demand Constraint of D2 from P1 and P2
        {'type': 'ineq', 'fun': lambda x: 2905/np.sum(x[10:12],axis=0)-0.8}, #Lower Limit of Fill rate
        #{'type': 'ineq', 'fun': lambda x: x[10]+x[11]-400}, #Higher Limit of Fill rate
        {'type': 'ineq', 'fun': lambda x: -x[6]-x[8]+x[10]}, #Production of P1 should exceed distribution from P1
        {'type': 'ineq', 'fun': lambda x: -x[7]-x[9]+x[11]}, #Production of P2 should exceed distribution from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[0]+x[2]}, #Supply from S1 and S2 to P1 for C1 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[1]+x[3]}, #Supply from S1 and S2 to P2 for C1 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: -x[10]+x[4]}, #Supply from from S1 to P1 for C2 exceed Production from P1
        {'type': 'ineq', 'fun': lambda x: -x[11]+x[5]}, #Supply from from S1 to P2 for C2 exceed Production from P2
        {'type': 'ineq', 'fun': lambda x: x[0]}, #Decision Variable from S11 to P1 for C7
        {'type': 'ineq', 'fun': lambda x: x[1]}, #Decision Variable from S11 to P2 for C7
        {'type': 'ineq', 'fun': lambda x: x[2]}, #Decision Variable from S1 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[3]}, #Decision Variable from S1 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[4]}, #Decision Variable from S6 to P1 for C3
        {'type': 'ineq', 'fun': lambda x: x[5]}, #Decision Variable from S6 to P2 for C3
        {'type': 'ineq', 'fun': lambda x: x[6]}, #Decision Variable from S4 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[7]}, #Decision Variable from S4 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[8]}, #Decision Variable from S7 to P1 for C4
        {'type': 'ineq', 'fun': lambda x: x[9]}, #Decision Variable from S7 to P2 for C4
        {'type': 'ineq', 'fun': lambda x: x[10]}, #Decision Variable from S9 to P1 for C5
        {'type': 'ineq', 'fun': lambda x: x[11]}, #Decision Variable from S9 to P2 for C5
        {'type': 'ineq', 'fun': lambda x: x[12]-0.8}, #Covid Variable from S1 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[13]-0.8}, #Covid Variable from S1 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[14]-0.8}, #Covid Variable from S2 to P1 for C1
        {'type': 'ineq', 'fun': lambda x: x[15]-0.8}, #Covid Variable from S2 to P2 for C1
        {'type': 'ineq', 'fun': lambda x: x[16]-0.8}, #Covid Variable from S3 to P1 for C2
        {'type': 'ineq', 'fun': lambda x: x[17]-0.8}, #Covid Variable from S3 to P2 for C2
        {'type': 'ineq', 'fun': lambda x: x[18]-0.8}, #Covid Variable from P1 to D1 
        {'type': 'ineq', 'fun': lambda x: x[19]-0.8}, #Covid Variable from P2 to D1 
        {'type': 'ineq', 'fun': lambda x: x[20]-0.8}, #Covid Variable from P1 to D2
        {'type': 'ineq', 'fun': lambda x: x[21]-0.8}, #Covid Variable from P2 to D2
        {'type': 'ineq', 'fun': lambda x: x[22]-0.8}, #Covid Variable for P1
        {'type': 'ineq', 'fun': lambda x: x[23]-0.8}, #Covid Variable from P2
        )


from psopy import init_feasible
# print(capacity.shape)

# vectorized implimentation---------------

x0=init_feasible(constraints=cons,low=low,high=high, shape=(100,24),max_retries=2000)
print(x0)
res = minimize(fun, x0, args=a, constraints=cons,options={'g_rate': 1., 'l_rate': 1., 'max_velocity': 4.,'stable_iter': 10000,'savefile':'Result0.8.csv'})
print(res)
init_feasible
print('quants:\n')
print(res.x.reshape((2,12)))
q=res.x.reshape(2,12)
print('qunatity from plant 1:',q[0,],'\n quantity from plant 2:',q[1,])
